In [ ]:
import mne
from mne.preprocessing import ICA
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration dictionary for easy parameter adjustment
CONFIG = {
    'raw_dir': Path('../../raw'),
    'output_dir': Path('processed'),
    'file_pattern': r'^sub-[A-Za-z0-9]+_task-DespicableMe_eeg\.set$',
    'notch_freq': 60.0,
    'bandpass_low': 1.0,  # Hz
    'bandpass_high': 40.0,  # Hz
    'n_components': 20,  # ICA components
    'ica_method': 'infomax',
    'artifact_threshold_factor': 2.0, 
    'variance_threshold': 5.0,
    'eog_threshold': 3.0, 
    'plot_channels': 20,  # Number of channels to plot
    'window_size': 5.0,  # Seconds for artifact window search
}

def setup_matplotlib_backend():
    """Make sure matplotlib is operating with interactive plots"""
    try:
        import matplotlib
        if not matplotlib.get_backend().lower().startswith(('tkagg', 'qt5agg', 'wxagg')):
            matplotlib.use('TkAgg', warn=False)
    except Exception as e:
        logging.warning(f"Failed to set matplotlib backend: {e}. Using default.")

def load_eeg_data(raw_dir, file_pattern):
    """Load EEG data matching the specified pattern."""
    raw_data = []
    raw_dir = Path(raw_dir)
    if not raw_dir.exists():
        raise FileNotFoundError(f"Raw data directory not found: {raw_dir}")
    
    for fname in raw_dir.iterdir():
        if re.match(file_pattern, fname.name):
            logging.info(f"Loading: {fname.name}")
            try:
                raw = mne.io.read_raw_eeglab(fname, preload=True)
                raw_data.append(raw)
            except Exception as e:
                logging.error(f"Failed to load {fname.name}: {e}")
    
    if not raw_data:
        raise ValueError(f"No files found matching pattern: {file_pattern}")
    
    if len(raw_data) > 1:
        logging.warning(f"Multiple files found. Using the first one: {raw_data[0].filenames[0]}")
    
    return raw_data[0]

def inspect_data(raw, title, n_channels, start=0.0, block=True):
    """Plot data for inspection."""
    logging.info(f"Displaying {title}. Close plot to continue...")
    fig = raw.plot(n_channels=n_channels, scalings='auto', title=title,
                   start=start, show_scrollbars=True, block=block)
    plt.show(block=block)
    return fig

def apply_filters(raw, notch_freq, bandpass_low, bandpass_high):
    """Apply notch and bandpass filters."""
    logging.info(f"Applying {notch_freq} Hz notch filter...")
    raw.notch_filter(freqs=[notch_freq], fir_design='firwin')
    
    logging.info(f"Applying {bandpass_low}-{bandpass_high} Hz bandpass filter...")
    raw.filter(bandpass_low, bandpass_high, fir_design='firwin')
    logging.info("Filtering complete!")

def interpolate_bad_channels(raw):
    """Interpolate bad channels if any are marked."""
    if raw.info['bads']:
        logging.info(f"Interpolating {len(raw.info['bads'])} bad channels: {raw.info['bads']}")
        raw.interpolate_bads(reset_bads=True)
        logging.info("Interpolation complete!")
    else:
        logging.info("No bad channels to interpolate.")

def detect_artifacts(raw, threshold_factor):
    """Detect extreme amplitude artifacts."""
    data = raw.get_data()
    ptp = np.ptp(data, axis=1)
    threshold = np.percentile(ptp, 95) * threshold_factor
    
    logging.info(f"Max peak-to-peak amplitude: {ptp.max() * 1e6:.1f} µV")
    logging.info(f"Threshold: {threshold * 1e6:.1f} µV")
    
    if ptp.max() > threshold:
        logging.warning("High amplitude artifacts detected! Consider manual segment rejection.")

def run_ica(raw, n_components, method):
    """Run ICA decomposition."""
    n_components = min(n_components, len(raw.ch_names) - 1)
    logging.info(f"Running ICA with {n_components} components using {method}...")
    ica = ICA(n_components=n_components, random_state=97, max_iter='auto', method=method)
    ica.fit(raw)
    logging.info("ICA fitting complete!")
    return ica

def select_ica_components(ica, raw):
    """Interactively select ICA components for exclusion."""
    logging.info("Displaying ICA components for interactive selection...")
    logging.info("Click components to exclude. Close window when done.")
    ica.plot_components(inst=raw, show=True)
    plt.show(block=True)
    
    if ica.exclude:
        logging.info(f"Components marked for exclusion: {ica.exclude}")
        logging.info("Showing time courses of excluded components...")
        ica.plot_properties(raw, picks=ica.exclude)
        plt.show(block=True)
    else:
        logging.info("No components marked for exclusion.")

    # Automatic EOG detection
    try:
        eog_indices, eog_scores = ica.find_bads_eog(raw, threshold=CONFIG['eog_threshold'])
        if eog_indices:
            logging.info(f"Automatically detected EOG components: {eog_indices}")
            logging.info(f"Scores: {[f'{s:.2f}' for s in eog_scores[eog_indices]]}")
            ica.exclude = list(set(ica.exclude + eog_indices))
    except Exception as e:
        logging.info(f"Automatic EOG detection failed: {e}")

    # High variance component detection
    try:
        sources = ica.get_sources(raw).get_data()
        component_var = np.var(sources, axis=1)
        total_var = np.sum(component_var)
        component_var_pct = (component_var / total_var) * 100
        high_var_components = [i for i, var in enumerate(component_var_pct) if var > CONFIG['variance_threshold']]
        if high_var_components:
            logging.info(f"Components with >{CONFIG['variance_threshold']}% variance: {high_var_components}")
            for i in high_var_components:
                logging.info(f"  Component {i}: {component_var_pct[i]:.1f}%")
    except Exception as e:
        logging.warning(f"Failed to calculate component variance: {e}")

    return ica

def review_ica_exclusions(ica, raw):
    """Review and modify ICA component exclusions."""
    logging.info("Reviewing ICA component exclusions...")
    logging.info("""
    📋 TIPS FOR IDENTIFYING ARTIFACTS:
    - EXCLUDE: Frontal spikes (eye blinks), lateral frontal drifts (eye movements),
               high-frequency temporal (muscle), single-channel (electrode artifacts).
    - KEEP: Posterior 8-13 Hz (alpha), frontal/central 4-8 Hz (theta),
            diffuse continuous (brain signals), high-variance brain-like patterns.
    """)
    
    if ica.exclude:
        logging.info(f"Currently marked for exclusion: {ica.exclude}")
        ica.plot_properties(raw, picks=ica.exclude)
        plt.show(block=True)
        
        while True:
            confirm = input("Keep these exclusions? (y/n): ").lower()
            if confirm in ['y', 'n']:
                break
            logging.warning("Invalid input. Please enter 'y' or 'n'.")
        
        if confirm == 'n':
            while True:
                modify_choice = input("Enter (a) to add, (r) to remove, (n) for new list, (c) to cancel: ").lower()
                if modify_choice not in ['a', 'r', 'n', 'c']:
                    logging.warning("Invalid choice. Please enter 'a', 'r', 'n', or 'c'.")
                    continue
                
                try:
                    if modify_choice == 'a':
                        add_components = input("Enter components to ADD (comma-separated, e.g., 3,7): ")
                        if add_components.strip():
                            new_comps = [int(x.strip()) for x in add_components.split(',')]
                            ica.exclude = list(set(ica.exclude + new_comps))
                    elif modify_choice == 'r':
                        remove_components = input("Enter components to REMOVE (comma-separated): ")
                        if remove_components.strip():
                            remove_comps = [int(x.strip()) for x in remove_components.split(',')]
                            ica.exclude = [c for c in ica.exclude if c not in remove_comps]
                    elif modify_choice == 'n':
                        new_components = input("Enter NEW list of components (comma-separated): ")
                        ica.exclude = [int(x.strip()) for x in new_components.split(',')] if new_components.strip() else []
                    else:  # cancel
                        break
                    logging.info(f"Updated exclusions: {ica.exclude}")
                    if ica.exclude:
                        ica.plot_properties(raw, picks=ica.exclude)
                        plt.show(block=True)
                    break
                except ValueError as e:
                    logging.warning(f"Invalid component numbers: {e}. Please enter integers.")
    else:
        add_manual = input("Manually specify components to exclude? (y/n): ").lower()
        if add_manual == 'y':
            while True:
                try:
                    components_to_exclude = input("Enter components to exclude (comma-separated, e.g., 0,2,5): ")
                    if components_to_exclude.strip():
                        ica.exclude = [int(x.strip()) for x in components_to_exclude.split(',')]
                        logging.info(f"Components to exclude: {ica.exclude}")
                        ica.plot_properties(raw, picks=ica.exclude)
                        plt.show(block=True)
                    break
                except ValueError as e:
                    logging.warning(f"Invalid component numbers: {e}. Please enter integers.")

    return ica

def apply_ica(raw, raw_before_ica, ica):
    """Apply ICA to remove selected components."""
    if not ica.exclude:
        logging.warning("No components marked for exclusion. Data will be unchanged.")
        proceed = input("Go back and exclude components? (y/n): ").lower()
        if proceed == 'y':
            logging.info("Please re-run component selection.")
            raise SystemExit("Stopping to allow component exclusion.")
    
    logging.info(f"Removing {len(ica.exclude)} component(s): {ica.exclude}")
    ica.apply(raw)
    logging.info("ICA applied successfully!")
    
    # Verify data change
    data_before = raw_before_ica.get_data()
    data_after = raw.get_data()
    difference = np.sum(np.abs(data_before - data_after))
    
    if difference < 1e-10:
        logging.warning("Data appears unchanged after ICA! Check component exclusions.")
    else:
        logging.info(f"Data modified (total absolute change: {difference:.2e})")
        var_before = np.var(data_before)
        var_after = np.var(data_after)
        var_change = ((var_after - var_before) / var_before) * 100
        logging.info(f"Variance before ICA: {var_before:.2e}")
        logging.info(f"Variance after ICA: {var_after:.2e}")
        logging.info(f"Variance change: {var_change:+.2f}%")
        
        if var_change > 10:
            logging.warning("Variance INCREASED significantly! Possible removal of brain signals.")
        elif var_change < -10:
            logging.info("Variance decreased - likely removed artifacts.")
        else:
            logging.info("Minor variance change - check overlay plot.")

def find_artifact_window(raw, window_size):
    """Find a time window with high variance for visualization."""
    data = raw.get_data()
    window_samples = int(window_size * raw.info['sfreq'])
    variances = []
    
    for i in range(0, data.shape[1] - window_samples, window_samples):
        var = np.var(data[:, i:i+window_samples])
        variances.append((i / raw.info['sfreq'], var))
    
    if variances:
        variances.sort(key=lambda x: x[1], reverse=True)
        max_start_time = max(0, raw.times[-1] - 10)
        return min(variances[0][0], max_start_time)
    return 0.0

def save_cleaned_data(raw, ica, output_dir, original_fname):
    """Save cleaned data and ICA solution."""
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    output_fname = original_fname.replace('.set', '_clean.fif')
    output_path = output_dir / output_fname
    raw.save(output_path, overwrite=True)
    logging.info(f"Cleaned data saved to: {output_path}")
    
    ica_fname = original_fname.replace('.set', '_ica.fif')
    ica_path = output_dir / ica_fname
    ica.save(ica_path, overwrite=True)
    logging.info(f"ICA solution saved to: {ica_path}")

def main():
    """Main preprocessing pipeline."""
    try:
        # Setup
        setup_matplotlib_backend()
        
        # Step 1: Load data
        print("=" * 60)
        print("STEP 1: LOADING DATA")
        print("=" * 60)
        raw = load_eeg_data(CONFIG['raw_dir'], CONFIG['file_pattern'])
        logging.info(f"Data loaded: {raw.n_times} samples, {len(raw.ch_names)} channels")
        logging.info(f"Sampling rate: {raw.info['sfreq']} Hz")
        logging.info(f"Duration: {raw.times[-1]:.2f} seconds")
        
        # Step 2: Initial inspection
        print("\n" + "=" * 60)
        print("STEP 2: INITIAL DATA INSPECTION")
        print("=" * 60)
        inspect_data(raw, "Raw Data (Unfiltered)", CONFIG['plot_channels'])
        
        # Step 3: Apply filters
        print("\n" + "=" * 60)
        print("STEP 3: APPLYING FILTERS")
        print("=" * 60)
        apply_filters(raw, CONFIG['notch_freq'], CONFIG['bandpass_low'], CONFIG['bandpass_high'])
        
        # Step 4: Inspect filtered data & mark bad channels
        print("\n" + "=" * 60)
        print("STEP 4: MARK BAD CHANNELS")
        print("=" * 60)
        print("Instructions: Click channel names to mark as bad (highlighted in red). Close plot when done.")
        inspect_data(raw, "Filtered Data - Click Channel Names to Mark as Bad", CONFIG['plot_channels'])
        
        # Step 5: Interpolate bad channels
        interpolate_bad_channels(raw)
        
        # Step 6: Set reference
        print("\n" + "=" * 60)
        print("STEP 6: SETTING AVERAGE REFERENCE")
        print("=" * 60)
        raw.set_eeg_reference('average', projection=True)
        raw.apply_proj()
        logging.info("Reference set to average!")
        
        # Step 7: Save pre-ICA state
        print("\n" + "=" * 60)
        print("STEP 7: SAVING PRE-ICA STATE")
        print("=" * 60)
        raw_before_ica = raw.copy()
        logging.info("Copy of filtered data created!")
        
        # Step 8: Artifact rejection
        print("\n" + "=" * 60)
        print("STEP 8: ARTIFACT REJECTION")
        print("=" * 60)
        detect_artifacts(raw, CONFIG['artifact_threshold_factor'])
        
        # Step 9: Run ICA
        print("\n" + "=" * 60)
        print("STEP 9: RUNNING ICA")
        print("=" * 60)
        ica = run_ica(raw, CONFIG['n_components'], CONFIG['ica_method'])
        
        # Step 10: Select ICA components
        print("\n" + "=" * 60)
        print("STEP 10: INTERACTIVE ICA COMPONENT SELECTION")
        print("=" * 60)
        ica = select_ica_components(ica, raw)
        
        # Step 11: Review ICA exclusions
        print("\n" + "=" * 60)
        print("STEP 11: REVIEW COMPONENT EXCLUSIONS")
        print("=" * 60)
        ica = review_ica_exclusions(ica, raw)
        
        # Step 12: Apply ICA
        print("\n" + "=" * 60)
        print("STEP 12: APPLYING ICA")
        print("=" * 60)
        apply_ica(raw, raw_before_ica, ica)
        
        # Step 13: Compare before/after
        print("\n" + "=" * 60)
        print("STEP 13: COMPARE BEFORE/AFTER ICA")
        print("=" * 60)
        artifact_time = find_artifact_window(raw_before_ica, CONFIG['window_size'])
        logging.info(f"Showing time window starting at {artifact_time:.1f} seconds (likely has artifacts)")
        inspect_data(raw_before_ica, "BEFORE ICA Cleaning (Filtered Only)", CONFIG['plot_channels'], start=artifact_time, block=False)
        inspect_data(raw, "AFTER ICA Cleaning (Artifacts Removed)", CONFIG['plot_channels'], start=artifact_time)
        
        # Step 14: Visualize removed components
        if ica.exclude:
            print("\n" + "=" * 60)
            print("STEP 14: VISUALIZE WHAT WAS REMOVED")
            print("=" * 60)
            ica.plot_overlay(raw_before_ica, exclude=ica.exclude, title="ICA Artifact Removal Overlay")
            plt.show(block=True)
        
        # Step 15: Save cleaned data
        print("\n" + "=" * 60)
        print("STEP 15: SAVE CLEANED DATA")
        print("=" * 60)
        save_choice = input("Save cleaned data? (y/n): ").lower()
        if save_choice == 'y':
            save_cleaned_data(raw, ica, CONFIG['output_dir'], os.path.basename(raw.filenames[0]))
        else:
            logging.info("Data not saved.")
        
        # Final summary
        print("\n" + "=" * 60)
        print("PREPROCESSING COMPLETE!")
        print("=" * 60)
        logging.info(f"Channels processed: {len(raw.ch_names)}")
        logging.info(f"ICA components removed: {len(ica.exclude)}")
        logging.info(f"Final data duration: {raw.times[-1]:.2f} seconds")
        logging.info("Summary of what was removed:")
        if ica.exclude:
            logging.info(f"  - Component(s) {ica.exclude} were identified as artifacts and removed")
        else:
            logging.info("  - No components were removed")
            
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()

2025-10-27 13:35:19,848 - INFO - Loading: sub-NDARAA396TWZ_task-DespicableMe_eeg.set
2025-10-27 13:35:19,983 - INFO - Loading: sub-NDARAC688ZM5_task-DespicableMe_eeg.set


STEP 1: LOADING DATA


C:\Users\JINQI\AppData\Local\Temp\ipykernel_52292\3112418428.py:49: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(fname, preload=True)
2025-10-27 13:35:20,111 - INFO - Loading: sub-NDARAC923GPW_task-DespicableMe_eeg.set
C:\Users\JINQI\AppData\Local\Temp\ipykernel_52292\3112418428.py:49: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(fname, preload=True)
2025-10-27 13:35:20,241 - INFO - Loading: sub-NDARAD256AHU_task-DespicableMe_eeg.set
2025-10-27 13:35:20,363 - WARNING - Multiple files found. Using the first one: c:\Users\JINQI\Coursework\DATA495\notebooks\joseph-nb\..\..\raw\sub-NDARAA396TWZ_task-DespicableMe_eeg.set
2025-10-27 13:35:20,374 - INFO - Data loaded: 106518 samples, 129 channels
2025-10-27 13:35:20,375 - INFO - Sampli


STEP 2: INITIAL DATA INSPECTION
Channels marked as bad:
none


2025-10-27 13:35:24,476 - INFO - Applying 60.0 Hz notch filter...


...

Channels marked as bad:
none
Channels marked as bad:
none

STEP 14: VISUALIZE WHAT WAS REMOVED
Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 3 ICA components
    Projecting back using 129 PCA components

STEP 15: SAVE CLEANED DATA
Writing c:\Users\JINQI\Coursework\DATA495\notebooks\joseph-nb\processed\sub-NDARAA396TWZ_task-DespicableMe_eeg_clean.fif
Closing c:\Users\JINQI\Coursework\DATA495\notebooks\joseph-nb\processed\sub-NDARAA396TWZ_task-DespicableMe_eeg_clean.fif
[done]


C:\Users\JINQI\AppData\Local\Temp\ipykernel_52292\3112418428.py:278: RuntimeWarning: This filename (c:\Users\JINQI\Coursework\DATA495\notebooks\joseph-nb\processed\sub-NDARAA396TWZ_task-DespicableMe_eeg_clean.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(output_path, overwrite=True)
2025-10-27 13:38:13,382 - INFO - Cleaned data saved to: processed\sub-NDARAA396TWZ_task-DespicableMe_eeg_clean.fif


Writing ICA solution to c:\Users\JINQI\Coursework\DATA495\notebooks\joseph-nb\processed\sub-NDARAA396TWZ_task-DespicableMe_eeg_ica.fif...


2025-10-27 13:38:13,388 - INFO - ICA solution saved to: processed\sub-NDARAA396TWZ_task-DespicableMe_eeg_ica.fif
2025-10-27 13:38:13,389 - INFO - Channels processed: 129
2025-10-27 13:38:13,389 - INFO - ICA components removed: 3
2025-10-27 13:38:13,389 - INFO - Final data duration: 213.03 seconds
2025-10-27 13:38:13,390 - INFO - Summary of what was removed:
2025-10-27 13:38:13,390 - INFO -   - Component(s) [1, 4, 6] were identified as artifacts and removed



PREPROCESSING COMPLETE!
